In [58]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
import torchvision.transforms as transforms
from PIL import Image, UnidentifiedImageError
import numpy as np

# Initialize the MTCNN model
mtcnn = MTCNN(keep_all=True, post_process=False)

# resnet = InceptionResnetV1(pretrained='casia-webface').eval()

# Set the model to evaluation mode
mtcnn.eval()


MTCNN(
  (pnet): PNet(
    (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (prelu1): PReLU(num_parameters=10)
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv2): Conv2d(10, 16, kernel_size=(3, 3), stride=(1, 1))
    (prelu2): PReLU(num_parameters=16)
    (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (prelu3): PReLU(num_parameters=32)
    (conv4_1): Conv2d(32, 2, kernel_size=(1, 1), stride=(1, 1))
    (softmax4_1): Softmax(dim=1)
    (conv4_2): Conv2d(32, 4, kernel_size=(1, 1), stride=(1, 1))
  )
  (rnet): RNet(
    (conv1): Conv2d(3, 28, kernel_size=(3, 3), stride=(1, 1))
    (prelu1): PReLU(num_parameters=28)
    (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv2): Conv2d(28, 48, kernel_size=(3, 3), stride=(1, 1))
    (prelu2): PReLU(num_parameters=48)
    (pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv3): Conv2d(48, 64,

In [59]:
image = Image.open("unnamed.jpg")
# Detect faces in the image
boxes, probs = mtcnn.detect(image)

# # If faces are detected, extract embeddings
# if boxes is not None:
#     aligned = mtcnn(image)
#     embeddings = resnet(aligned).detach()

In [60]:
from PIL import ImageDraw

img = Image.open("unnamed.jpg") 
# If faces are detected, 'boxes' will contain the bounding box coordinates
if boxes is not None:
    for box in boxes:
        # Draw bounding boxes on the image
        draw = ImageDraw.Draw(img)
        draw.rectangle(box.tolist(), outline='red', width=3)

# Display or save the image with detected faces
img.show()  

In [40]:
img.size

(379, 512)

In [38]:
probs

array([0.999933123588562], dtype=object)

In [25]:
import cv2

original_image = cv2.imread("unnamed.jpg")

# Set device to CPU
device = torch.device("cpu")

# Convert image to tensor
image_tensor = torch.from_numpy(original_image).permute(2, 0, 1).float() / 255.0
image_tensor = image_tensor.unsqueeze(0).to(device)
image_tensor.requires_grad = True


[DEBUG] Initial image_tensor grad_fn: None


In [26]:
# Forward pass through MTCNN
outputs = mtcnn.detect(image, landmarks=False)  # Disable landmarks if not needed

if outputs[0] is None or len(outputs[0]) == 0:
    print("No faces detected!")
    exit()


# Define a dummy loss (e.g., targeting a specific bounding box value)
target_bbox = torch.tensor([[30, 30, 130, 130]])  # Example target bounding box
loss_fn = torch.nn.MSELoss()

# Compute the loss
predicted_bbox = torch.tensor(outputs[0])  # Assuming outputs[0] contains bounding boxes
loss = loss_fn(predicted_bbox, target_bbox)

# Backpropagation to compute gradients
loss.backward()

# Perform FGSM attack
epsilon = 0.01  # Define perturbation magnitude
data_grad = image.grad.data
perturbed_image = fgsm_attack(image, epsilon, data_grad)

RuntimeError: torch.cat(): expected a non-empty list of Tensors